<a href="https://colab.research.google.com/github/Madhumitha14/Color-identification/blob/main/TableDetAndStructGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google Drive Authentication and Mounting

In [ ]:
# Mounting the Drive
from google.colab import drive

# Authorization
drive.mount('/content/drive')

!ls '/content/drive/My Drive/project/data/Marmot dataset/English/'
myDrive = '/content/drive/My Drive/project/data/Marmot dataset/English/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Negative  Positive


Imports

In [ ]:
import os
import torch
import numpy as np
import struct
from PIL import Image
import xml.etree.ElementTree as ET
import glob

Defining paths


In [ ]:
# Defining data paths
pos_data_path = myDrive + 'Positive/' 
neg_data_path = myDrive + 'Negative/'

# For images
pos_img_path = pos_data_path + 'Raw' + '/*.bmp'
neg_img_path = neg_data_path + 'Raw' + '/*.bmp'

# For table coordinates
pos_table_xml_path = pos_data_path + 'Labeled/'
neg_table_xml_path = neg_data_path + 'Labeled/'

#For processed data
Processed_data = myDrive = '/content/drive/My Drive/project/data/Marmot dataset/English/Positive/'+'Marmot processed'
image_path = Processed_data + '/image/'
table_mask_path = Processed_data + '/table_mask'


Pre-processing

In [ ]:
 # Preprocessing
def get_table_bbox(table_xml_path, new_image_shape):
  tree = ET.parse(table_xml_path)
  root = tree.getroot()

  left,top,right,bottom = list(map(lambda x: struct.unpack('!d', bytes.fromhex(x))[0], root.get("CropBox").split()))
  width = abs(right - left)
  height = abs(top - bottom)
  table_bounding_boxes = []
  
  for table in root.findall(".//Composite[@Label='TableBody']"):
    print(table)
    x0in, y0in, x1in, y1in  = list(map(lambda x: struct.unpack('!d', bytes.fromhex(x))[0], table.get("BBox").split()))
    x0 = round(new_image_shape[1] * (x0in - left) / width)
    x1 = round(new_image_shape[1] * (x1in - left) / width)
    y0 = round(new_image_shape[0] * (top - y0in) / height)
    y1 = round(new_image_shape[0] * (top - y1in) / height)
    table_bounding_boxes.append([x0, y0, x1, y1])
  return table_bounding_boxes



#Create mask

def create_mask(new_h, new_w, bboxes = None ):
  mask = np.zeros((new_h,new_w), dtype = np.int32)
  if bboxes is None or len(bboxes)==0:
    return Image.fromarray(mask)
  for box in bboxes:
    mask[box[1]:box[3], box[0]:box[2]] = 255
  return Image.fromarray(mask)

In [90]:
processed_data = []

new_h, new_w = 1024, 1024

neg_img_path = glob.glob(neg_img_path)
pos_img_path = glob.glob(pos_img_path)

for i, data in enumerate([neg_img_path, pos_img_path]):

    for j, img_path in enumerate(data):
        
        image_name = os.path.basename(img_path)
        image = Image.open(img_path).convert('LA')
        w, h = image.size
        
        #convert to RGB image
        image = image.resize((new_h, new_w)).convert("RGB")
      
        table_bboxes = []
        
        if i == 1:
          #get xml filename
            xml_file = image_name.replace('bmp', 'xml')
            table_xml_path = os.path.join(pos_table_xml_path, xml_file)
        
           #get table boxes
            table_bboxes = get_table_bbox(table_xml_path, (new_h, new_w))
            
           #generate masks
            table_mask = create_mask(new_h, new_w, table_bboxes)
        
           #save images and masks
            save_image_path = os.path.join(image_path, image_name.replace('bmp', 'jpg'))
            save_table_mask_path = os.path.join(table_mask_path, image_name[:-4] + '_table_mask.png')
        
            print('image_name', image_name)
            print('save img path', save_image_path)
            print('save table mask path', save_table_mask_path)
        
            image.save(save_image_path)
            table_mask.save(save_table_mask_path)
        
        
          #add data to dataframe
            len_table = len(table_bboxes)
      

            value = (save_image_path, save_table_mask_path, h, w, int(len_table != 0), \
                 len_table, table_bboxes, )
        
            processed_data.append(value)

<Element 'Composite' at 0x7fbfe6da65c0>
image_name 10.1.1.1.2028_14.bmp
save img path /content/drive/My Drive/project/data/Marmot dataset/English/Positive/Marmot processed/image/10.1.1.1.2028_14.jpg
save table mask path /content/drive/My Drive/project/data/Marmot dataset/English/Positive/Marmot processed/table_mask/10.1.1.1.2028_14_table_mask.png
<Element 'Composite' at 0x7fbfe6cb03b0>
image_name 10.1.1.1.2020_8.bmp
save img path /content/drive/My Drive/project/data/Marmot dataset/English/Positive/Marmot processed/image/10.1.1.1.2020_8.jpg
save table mask path /content/drive/My Drive/project/data/Marmot dataset/English/Positive/Marmot processed/table_mask/10.1.1.1.2020_8_table_mask.png
image_name 10.1.1.1.2031_24.bmp
save img path /content/drive/My Drive/project/data/Marmot dataset/English/Positive/Marmot processed/image/10.1.1.1.2031_24.jpg
save table mask path /content/drive/My Drive/project/data/Marmot dataset/English/Positive/Marmot processed/table_mask/10.1.1.1.2031_24_table_mask.

KeyboardInterrupt: ignored

In [ ]:
#Dataloader
class ImageFolder(nn.Module):
    def __init__(self, df, isTrain = True, transform = None):
        super(ImageFolder, self).__init__()
        self.df = df
        if transform is None:
            self.transform = A.Compose([
                        #ToTensor --> Normalize(mean, std) 
                        A.Normalize(
                            mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225],
                            max_pixel_value = 255,
                        ),
                        ToTensorV2()
                    ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        img_path, table_mask_path, col_mask_path = self.df.iloc[index, 0], self.df.iloc[index, 1], self.df.iloc[index, 2]
        image = np.array(Image.open("../" + img_path))
        table_image = torch.FloatTensor(np.array(Image.open("../" + table_mask_path))/255.0).reshape(1,1024,1024)
        column_image = torch.FloatTensor(np.array(Image.open("../" + col_mask_path))/255.0).reshape(1,1024,1024)
        image = self.transform(image = image)['image']

        return {"image":image,"table_image":table_image, "column_image": column_image}
      
df = pd.read_csv(data_path)
train_data, test_data  = train_test_split(df, test_size = 0.2, random_state = config.SEED, stratify = df.hasTable)

train_dataset = ImageFolder(train_data, isTrain = True, transform = None)
test_dataset = ImageFolder(test_data, isTrain = False, transform = None)

train_loader =  DataLoader(train_dataset, batch_size = config.BATCH_SIZE, shuffle=True, num_workers = 4, pin_memory=True)
test_loader =  DataLoader(test_dataset, batch_size = config.BATCH_SIZE, shuffle=False, num_workers = 4, pin_memory=True)
